In [29]:
#!pip install streamlit

     ---------------------------------------- 8.1/8.1 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 4.8/4.8 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 195.4/195.4 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 24.9/24.9 MB 2.7 MB/s eta 0:00:00
     -------------------------------------- 996.9/996.9 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB ? eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1


In [1]:
import random
#import streamlit as st
# Data Matakuliah
matakuliah = {
    1: {"MII225201": "Analisis Algoritme", "hari": 2, "jam": 2},
    2: {"MII225202": "Matematika untuk Ilmu Komputer", "hari": 4, "jam": 3},
    3: {"MII226001": "Metodologi Riset Ilmu Komputer", "hari": 5, "jam": 3},
    4: {"MII225201": "Pengolahan dan Analisis Citra Digital", "hari": 4, "jam": 3},
    5: {"MII226502": "Sistem Temu Balik Informasi", "hari": 4, "jam": 1},
    6: {"MII226503": "Data Warehouse dan Inteligensi Bisnis", "hari": 2, "jam": 1},
    7: {"MII226504": "Pengembangan Perangkat Lunak", "hari": 3, "jam": 3},
    8: {"MII226505": "Kecerdasan Digital dan Informatika Sosial", "hari": 1, "jam": 3},
    9: {"MII226506": "Manajemen dan Audit Sistem Informasi", "hari": 1, "jam": 3},
    10: {"MII226203": "Teori Komputasi", "hari": 4, "jam": 2},
    11: {"MII226401": "Kecerdasan Komputasional dan Pembelajaran Mesin", "hari": 2, "jam": 3},
    12: {"MII226402": "Rekayasa Fitur dan Pengenalan Pola", "hari": 2, "jam": 2},
    13: {"MII226404": " Sistem Pendukung Pembuatan Keputusan ", "hari": 4, "jam": 2},
    14: {"MII226501": " Data Science ", "hari": 3, "jam": 2},
    15: {"MII226601": " Komputasi Awan dan Keamanan Siber ", "hari": 2, "jam": 3},
    16: {"MII226602": " Jaringan Komputer Lanjut ", "hari": 3, "jam": 2},
    17: {"MII226603": " Platform dan Arsitektur Big Data ", "hari": 2, "jam": 2},
}

# Kode Ruang
kode_ruang = {419: 1, 420: 2, 421: 3, 422: 4}

# Parameter Algoritma Genetika
POPULATION_SIZE = 10
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
NUM_MATKUL = 6

# Fungsi untuk menghitung nilai fitness
def calculate_fitness(individual):
    # Hitung penalti untuk setiap individu
    penalty = 0
    hari_jam_pairs = set([(matakuliah[matkul]["hari"], matakuliah[matkul]["jam"]) for matkul in individual])
    for hari, jam in hari_jam_pairs:
        count_matkul_hari_jam = sum(1 for matkul in individual if matakuliah[matkul]["hari"] == hari and matakuliah[matkul]["jam"] == jam)
        if count_matkul_hari_jam > 1:  # Soft Constraint: Mahasiswa mengikuti maksimal 1 mata kuliah dalam 1 hari dan jam yang sama
            penalty += count_matkul_hari_jam - 1
    
    for sesi in set([matakuliah[matkul]["jam"] for matkul in individual]):
        if sesi:
            count_dosen_sesi = sum(1 for matkul in individual if matakuliah[matkul]["jam"] == jam)
            if count_dosen_sesi > 2:  # Soft Constraint: Dosen mengajar 2 Matakuliah dalam 1 sesi
                penalty += count_dosen_sesi - 2
    
    # Hitung fitness
    fitness = 1 / (1 + penalty)
    return fitness

# Fungsi untuk menghasilkan populasi awal
def generate_initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        individual = random.sample(list(matakuliah.keys()), NUM_MATKUL)
        population.append(individual)
    return population

# Fungsi seleksi menggunakan metode roulette wheel
def roulette_wheel_selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    r = random.uniform(0, total_fitness)
    cumulative_fitness = 0
    for i, fitness in enumerate(fitness_values):
        cumulative_fitness += fitness
        if cumulative_fitness >= r:
            return population[i]

# Fungsi crossover menggunakan 1-point crossover
def one_point_crossover(parent1, parent2):
    # crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:1] + parent2[1:]
    child2 = parent2[:1] + parent1[1:]
    # child1= parent
    return child1, child2

# Fungsi mutasi menggunakan pemilihan nilai acak

def mutate(individual):
    if random.random() < MUTATION_RATE:
        # Pilih dua matakuliah secara acak untuk ditukar
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

# Algoritma Genetika
        
def genetic_algorithm():
    # Inisialisasi populasi awal
    population = generate_initial_population()
    for generation in range(MAX_GENERATIONS):
        # Menghitung fitness untuk setiap individu dalam populasi
        fitness_values = [calculate_fitness(individual) for individual in population]
        # Memilih individu untuk reproduksi (seleksi)
        new_population = []
        for _ in range(POPULATION_SIZE):
            parent1 = roulette_wheel_selection(population, fitness_values)
            parent2 = roulette_wheel_selection(population, fitness_values)
            child1, child2 = one_point_crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.extend([child1, child2])
        population = new_population
  
    # Mengambil individu dengan nilai fitness tertinggi
    best_individual = max(population, key=calculate_fitness)
    best_fitness = calculate_fitness(best_individual)

    return best_individual, best_fitness

#Main program

if __name__ == "__main__":
    best_solution, best_fitness = genetic_algorithm()
    print("Jadwal Optimal:")
    for matkul in best_solution:
        print(f"Matakuliah: {matkul}, Hari: {matakuliah[matkul]['hari']}, Jam: {matakuliah[matkul]['jam']}, 
              Ruang: {kode_ruang[random.choice(list(kode_ruang.keys()))]}")
    print("Fitness Terbaik:", best_fitness)

# def main():
#     st.title("Optimasi Jadwal Mata Kuliah dengan Algoritma Genetika")

#     best_solution, best_fitness = genetic_algorithm()

#     st.header("Jadwal Optimal:")
#     for matkul in best_solution:
#         st.write(f"Matakuliah: {matkul}, Hari: {matakuliah[matkul]['hari']}, Jam: {matakuliah[matkul]['jam']}, Ruang: {kode_ruang[random.choice(list(kode_ruang.keys()))]}")
#     st.header("Fitness Terbaik:")
#     st.write(best_fitness)

# if __name__ == "__main__":
#   main()


Jadwal Optimal:
Matakuliah: 16, Hari: 3, Jam: 2, Ruang: 1
Matakuliah: 13, Hari: 4, Jam: 2, Ruang: 3
Matakuliah: 2, Hari: 4, Jam: 3, Ruang: 4
Matakuliah: 6, Hari: 2, Jam: 1, Ruang: 4
Matakuliah: 11, Hari: 2, Jam: 3, Ruang: 2
Matakuliah: 5, Hari: 4, Jam: 1, Ruang: 4
Fitness Terbaik: 1.0


In [12]:
# Main program
if __name__ == "__main__":
    best_solution, best_fitness = genetic_algorithm()
    print("Jadwal Optimal:")
    for matkul in best_solution:
        print(f"Matakuliah: {matkul}, Hari: {matakuliah[matkul]['hari']}, Jam: {matakuliah[matkul]['jam']}, Ruang: {kode_ruang[random.choice(list(kode_ruang.keys()))]}")
    print("Fitness Terbaik:", best_fitness)

[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12]]
[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12], [14, 10, 2, 1, 12, 6], [3, 14, 11, 3, 15, 16]]
[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12], [14, 10, 2, 1, 12, 6], [3, 14, 11, 3, 15, 16], [11, 8, 5, 2, 8, 12], [14, 3, 9, 15, 9, 3]]
[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12], [14, 10, 2, 1, 12, 6], [3, 14, 11, 3, 15, 16], [11, 8, 5, 2, 8, 12], [14, 3, 9, 15, 9, 3], [10, 11, 5, 16, 13, 4], [1, 12, 11, 14, 16, 12]]
[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12], [14, 10, 2, 1, 12, 6], [3, 14, 11, 3, 15, 16], [11, 8, 5, 2, 8, 12], [14, 3, 9, 15, 9, 3], [10, 11, 5, 16, 13, 4], [1, 12, 11, 14, 16, 12], [3, 14, 11, 1, 16, 17], [14, 2, 4, 10, 12, 6]]
[[10, 11, 1, 11, 10, 14], [3, 13, 5, 14, 16, 12], [14, 10, 2, 1, 12, 6], [3, 14, 11, 3, 15, 16], [11, 8, 5, 2, 8, 12], [14, 3, 9, 15, 9, 3], [10, 11, 5, 16, 13, 4], [1, 12, 11, 14, 16, 12], [3, 14, 11, 1, 16, 17], [14, 2, 4, 10, 12, 6], [1, 11, 5, 14, 16, 12], [10, 12, 11, 16, 13, 4]]